In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/datasets/bitcoin

/content/drive/MyDrive/Colab Notebooks/datasets/bitcoin


In [2]:
data_path = '/content/drive/MyDrive/Colab Notebooks/datasets/bitcoin/btc_4h_2017_2020.csv'

In [3]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 6.5MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437794 sha256=6e63d38d26eaabce9cf7f925aa23edf2883f4e8f20277300c98a897ef6a848f4
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib


In [4]:
!pip install fastai -Uq

     |████████████████████████████████| 194kB 5.7MB/s 
     |████████████████████████████████| 61kB 4.5MB/s 


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import talib
import fastai.tabular.all as ft

In [14]:
df = pd.read_csv(data_path)
columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'number_of_trades']
df = df[columns]
df['open_time'] = pd.to_datetime(df['open_time'])
# df = df.set_index('open_time')
df.tail()

,open_time,open,high,low,close,volume,number_of_trades
7518,2021-01-24 00:00:00,32078.00,32480.06,31630.00,32480.05,7766.196870,203631
7519,2021-01-24 04:00:00,32480.06,33071.00,32346.44,32801.15,11135.544522,242775
7520,2021-01-24 08:00:00,32797.21,33030.00,32465.00,32833.37,7600.854471,233158
7521,2021-01-24 12:00:00,32833.37,32931.59,31858.68,31957.05,9939.326135,275946
7522,2021-01-24 16:00:00,31956.90,32139.60,31500.00,31825.97,6364.327848,173282


# Create features using 

The the outputs by the pattern recognition functions in talib:

- +200 bullish pattern with confirmation
- +100 bullish pattern (most cases)
- 0 none
- -100 bearish pattern
- -200 bearish pattern with confirmation



In [24]:
def handle_dates(df, date_column='open_time', add_column=True):
    col = df[date_column]
    df = ft.add_datepart(df, date_column)
    if add_column:
        # add the column again
        df[date_column] = col

    return df

def add_rsi(df):
    rsi = talib.RSI(df.close)
    df['rsi'] = rsi

    return df
    
def add_macd(df):
    macd, macdsignal, macdhist = talib.MACD(df.close, fastperiod=12, slowperiod=26, signalperiod=9)

    df['macd'] = macd
    df['macdsignal'] = macdsignal
    df['macdhist'] = macdhist

    return df

def add_apo(df):
    apo = talib.APO(df.close, fastperiod=12, slowperiod=26, matype=0)
    df['apo'] = apo

    return df

def add_ema(df, period=200, volume=False):
    
    if volume:
        ema = talib.EMA(df.volume, timeperiod=period)
        df[f'ema_{period}_volume'] = ema
    else:
        ema = talib.EMA(df.close, timeperiod=period)      
        df[f'ema_{period}'] = ema

    return df

def add_bbands(df):
    upperband, middleband, lowerband = talib.BBANDS(df.close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    df['bb_upper'] = upperband
    df['bb_middleband'] = middleband
    df['bb_lower'] = lowerband

    return df

def add_psar(df):
    psar = talib.SAR(df.high, df.low, acceleration=0, maximum=0)
    df['parabolic_sar'] = psar

    return df

# Pattern recognition functions

def three_inside_up_down(df):
    up_down = talib.CDL3INSIDE(df.open, df.high, df.low, df.close)
    df['3inside_up_down'] = up_down

    return df

def three_line_strike(df):
    line_strike = talib.CDL3LINESTRIKE(df.open, df.high, df.low, df.close)
    df['3line_strike'] = line_strike

    return df

def three_stars_south(df):
    stars_south = talib.CDL3STARSINSOUTH(df.open, df.high, df.low, df.close)
    df['3stars_south'] = stars_south

    return df

def advancing_white_soldiers(df):
    advancing_soldiers = talib.CDL3WHITESOLDIERS(df.open, df.high, df.low, df.close)
    df['3advancing_white_soldiers'] = advancing_soldiers

    return df
    
def belt_hold(df):
    belt_hold = talib.CDLBELTHOLD(df.open, df.high, df.low, df.close)
    df['belt_hold'] = belt_hold

    return df

def breakaway(df):
    breakaway = talib.CDLBREAKAWAY(df.open, df.high, df.low, df.close)
    df['breakaway'] = breakaway

    return df

def closing_marubozu(df):
    marubozu = talib.CDLCLOSINGMARUBOZU(df.open, df.high, df.low, df.close)
    df['closing_marubozu'] = marubozu

    return df

def counteratack(df):
    counteratack = talib.CDLCOUNTERATTACK(df.open, df.high, df.low, df.close)
    df['counteratack'] = counteratack
    
    return df

def doji_star(df):
    doji_star = talib.CDLDOJISTAR(df.open, df.high, df.low, df.close)
    df['doji_star'] = doji_star

    return df

def dragonfly_doji(df):
    dragonfly = talib.CDLDRAGONFLYDOJI(df.open, df.high, df.low, df.close)
    df['dragonfly_doji'] = dragonfly

    return df

def engulfing(df):
    engulfing = talib.CDLENGULFING(df.open, df.high, df.low, df.close)
    df['engulfing'] = engulfing

    return df

def gravestone_doji(df):
    gravestone = talib.CDLGRAVESTONEDOJI(df.open, df.high, df.low, df.close)
    df['gravestone_doji'] = gravestone

    return df

def hammer(df):
    hammer = talib.CDLHAMMER(df.open, df.high, df.low, df.close)
    df['hammer'] = hammer

    return df

def hanging_man(df):
    hanging = talib.CDLHANGINGMAN(df.open, df.high, df.low, df.close)
    df['hanging_man'] = hanging

    return df

def inverted_hammer(df):
    inverted_hammer = talib.CDLINVERTEDHAMMER(df.open, df.high, df.low, df.close)
    df['inverted_hammer'] = inverted_hammer

    return df

def matching_low(df):
    low = talib.CDLMATCHINGLOW(df.open, df.high, df.low, df.close)
    df['matching_low'] = low

    return df

def morning_doji_star(df):
    morning = talib.CDLMORNINGDOJISTAR(df.open, df.high, df.low, df.close, penetration=0)
    df['morning_doji_star'] = morning

    return df

def separating_lines(df):
    lines = talib.CDLSEPARATINGLINES(df.open, df.high, df.low, df.close)
    df['separating_lines'] = lines
    
    return df

def shooting_star(df):
    shooting = talib.CDLSHOOTINGSTAR(df.open, df.high, df.low, df.close)
    df['shooting_star'] = shooting

    return df

def unique3river(df):
    river = talib.CDLUNIQUE3RIVER(df.open, df.high, df.low, df.close)
    df['unique3_river'] = river

    return df

In [25]:
df_copy = df.copy()
df_copy = handle_dates(df_copy)
df_copy = add_rsi(df_copy)
df_copy = add_macd(df_copy)
df_copy = add_apo(df_copy)
for ema in [20, 40, 50, 60, 200]:
    df_copy = add_ema(df_copy, ema)
for ema in [20, 40, 200]:
    df_copy = add_ema(df_copy, ema, volume=True)
df_copy = add_bbands(df_copy)
df_copy = add_psar(df_copy)
df_copy = three_inside_up_down(df_copy)
df_copy = three_line_strike(df_copy)
df_copy = three_stars_south(df_copy)
df_copy = advancing_white_soldiers(df_copy)
df_copy = belt_hold(df_copy)
df_copy = breakaway(df_copy)
df_copy = closing_marubozu(df_copy)
df_copy = counteratack(df_copy)
df_copy = doji_star(df_copy)
df_copy = dragonfly_doji(df_copy)
df_copy = engulfing(df_copy)
df_copy = gravestone_doji(df_copy)
df_copy = hammer(df_copy)
df_copy = hanging_man(df_copy)
df_copy = inverted_hammer(df_copy)
df_copy = matching_low(df_copy)
df_copy = morning_doji_star(df_copy)
df_copy = separating_lines(df_copy)
df_copy = shooting_star(df_copy)
df_copy = unique3river(df_copy)
df_copy.head(1)



,open,high,open_timeWeek,low,close,volume,number_of_trades,open_timeYear,open_timeMonth,open_timeDay,open_timeDayofweek,open_timeDayofyear,open_timeIs_month_end,open_timeIs_month_start,open_timeIs_quarter_end,open_timeIs_quarter_start,open_timeIs_year_end,open_timeIs_year_start,open_timeElapsed,open_time,rsi,macd,macdsignal,macdhist,apo,ema_20,ema_40,ema_50,ema_60,ema_200,ema_20_volume,ema_40_volume,ema_200_volume,bb_upper,bb_middleband,bb_lower,parabolic_sar,3inside_up_down,3line_strike,3stars_south,3advancing_white_soldiers,belt_hold,breakaway,closing_marubozu,counteratack,doji_star,dragonfly_doji,engulfing,gravestone_doji,hammer,hanging_man,inverted_hammer,matching_low,morning_doji_star,separating_lines,shooting_star,unique3_river
0,4261.48,4349.99,33,4261.32,4349.99,82.088865,334,2017,8,17,3,229,False,False,False,False,False,False,1502942400,2017-08-17 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
df_copy.to_csv('btc_feature_engineering', index=False)